In [ ]:
#mml_spark
from mmlspark.lightgbm import LightGBMClassifier, LightGBMClassificationModel
from pyspark.ml.feature import StringIndexer, VectorAssembler

model_path = "./model.lgb"
ft = ["ad_tag_id","device_id","domain_name","url_pattern2","slot_id","slideshow","seller_seller_tag","is_commercial_url_nb","is_commercial_url_lda","seller_adblock_id_misc","seller","cat0","cat1","cat2"]

model = LightGBMClassificationModel.loadNativeModelFromFile(model_path)
sdf = spark.read.parquet("./000.parquet")

features=[]
for f in ft:
    string_indexer = StringIndexer(inputCol=f, outputCol=f + "_index")
    model_si = string_indexer.fit(sdf)
    sdf = model_si.transform(sdf)
    features.append(f + "_index")

vector_assembler = VectorAssembler(inputCols=eatures, outputCol="features")
data = vector_assembler.transform(sdf)
preds = model.transform(sdf)

pred_probs = preds.select("probability").rdd.flatMap(lambda x: x).collect()
print(pred_probs)



#pandas (classic lightgbm)
import pandas as pd
import lightgbm as lgb

model = lgb.Booster(model_file = "./model.lgb")
df = pd.read_parquet("./000.parquet")

def set_column_dtype(df, col):
    temp = df[col].astype("str").astype("category").values
    del df[col]
    df[col] = temp
    
def set_dtypes(df, feature_list):
    for name in feature_list:
        set_column_dtype(df, name)
    return df

        
df = set_dtypes(df, ft)
df = df[ft]

preds = model.predict(df)
print(preds)